Importing libraries, loading environment variables from the .env file, accessing the API Key, and creating the Flask app

In [ ]:
from flask import Flask
from werkzeug.serving import run_simple
import os
from dotenv import load_dotenv
import tmdbsimple as tmdb

# Loading environment variables from the .env file
load_dotenv()

# Accessing the TMDb API key
TMDB_API_KEY = os.getenv("TMDB_API_KEY")
print(TMDB_API_KEY)

# Initializing the Flask app
app = Flask(__name__) # This creates an instance of the Flask class and assigns it to a variable. It's like creating a blueprint for the app

# Define's a route
@app.route("/") # This is a route decorator. It tells Flask which URL to connect to this function
def home(): # This function runs when someone visits the specified route
    print("Home route was accessed")
    return "Hello, Flask!"

# Define's a dynamic route for genres
@app.route("/genre/<genre_name>")
def show_genre(genre_name):
    print(f"Genre route was accessed with genre: {genre_name}")
    return f"You chose the {genre_name.capitalize()} genre!"

# Run the app
print("Starting Flask app...")
run_simple("localhost", 5000, app)

Fetching genre data from The Movie Database